In [65]:
import numpy as np
import pandas as pd

In [66]:
hydrate=pd.read_csv('input/s2-hydrate.csv',index_col=0)
cages=pd.read_csv('input/s2-cages.csv')
thf=pd.read_csv('input/thf.csv')

In [67]:
hydrate_ox=pd.DataFrame()
hydrate_ox[['X','Y','Z']]=hydrate[['OX','OY','OZ']]
hydrate_ox.insert(0,'ELEMENT','OW')
hydrate_hy=pd.DataFrame()
hydrate_hy[['X','Y','Z']]=hydrate[['H1X','H1Y','H1Z']]
hydrate_hy.insert(0, 'ELEMENT', 'HW1')
hydrate_hz=pd.DataFrame()
hydrate_hz[['X','Y','Z']]=hydrate[['H2X','H2Y','H2Z']]
hydrate_hz.insert(0, 'ELEMENT', 'HW2')

In [68]:
hydrate_total = pd.DataFrame(columns=['ELEMENT', 'X', 'Y', 'Z', 'RESIDUE_NUM'])
hydrate_total

,ELEMENT,X,Y,Z,RESIDUE_NUM


In [69]:
residue_num = 1
for row in range(len(hydrate_ox.index)):
    water = pd.concat([hydrate_ox.iloc[row], hydrate_hy.iloc[row], hydrate_hz.iloc[row]], axis=1, ignore_index=True).T
    water['RESIDUE_NUM'] = residue_num
    hydrate_total = pd.concat([hydrate_total, water], axis=0, ignore_index=True)
    residue_num += 1
hydrate_total

,ELEMENT,X,Y,Z,RESIDUE_NUM
0,OW,5.92487,5.92487,5.92487,1
1,HW1,5.75998,6.83792,5.68955,1
2,HW2,6.84272,5.77984,5.69518,1
3,OW,5.32179,5.32179,8.5662,2
4,HW1,5.53914,5.52632,7.65672,2
...,...,...,...,...,...
403,HW1,4.86407,13.52592,13.58037,135
404,HW2,3.79358,12.45544,13.57796,135
405,OW,12.98252,4.31665,12.9933,136
406,HW1,12.44593,3.72957,13.52588,136


In [70]:
hydrate_total['MOL'] = 'HOH'
hydrate_total

,ELEMENT,X,Y,Z,RESIDUE_NUM,MOL
0,OW,5.92487,5.92487,5.92487,1,HOH
1,HW1,5.75998,6.83792,5.68955,1,HOH
2,HW2,6.84272,5.77984,5.69518,1,HOH
3,OW,5.32179,5.32179,8.5662,2,HOH
4,HW1,5.53914,5.52632,7.65672,2,HOH
...,...,...,...,...,...,...
403,HW1,4.86407,13.52592,13.58037,135,HOH
404,HW2,3.79358,12.45544,13.57796,135,HOH
405,OW,12.98252,4.31665,12.9933,136,HOH
406,HW1,12.44593,3.72957,13.52588,136,HOH


In [71]:
large_cages = cages[cages['TYPE'].str.contains('Large')]
large_cages

,TYPE,X,Y,Z
16,Large 1,8.651,8.655,8.654
17,Large 2,4.322,4.327,12.984
18,Large 3,12.991,4.327,4.326
19,Large 4,4.319,12.983,4.326
20,Large 5,12.988,12.983,12.984
21,Large 6,0.004,0.000,8.655
22,Large 7,8.655,0.000,0.001
23,Large 8,0.000,8.655,0.000


In [72]:
thf_center = thf[['X', 'Y', 'Z']].mean()
thf_center

X   -0.357000
Y   -0.000231
Z    0.003769
dtype: float64

In [73]:
thf['RESIDUE_NUM'] = -1
thf['MOL'] = 'THF'
thf

,ELEMENT,X,Y,Z,RESIDUE_NUM,MOL
0,C00,0.290,-1.165,0.120,-1,THF
1,C01,-1.116,-0.712,-0.205,-1,THF
2,C02,-1.092,0.730,0.231,-1,THF
3,C03,0.317,1.148,-0.128,-1,THF
4,O04,1.139,-0.017,-0.012,-1,THF
5,H05,0.638,-1.958,-0.547,-1,THF
6,H06,0.364,-1.518,1.154,-1,THF
7,H07,-1.882,-1.310,0.296,-1,THF
8,H08,-1.286,-0.770,-1.287,-1,THF
9,H09,-1.234,0.792,1.317,-1,THF


In [74]:
# This loop recenters THF and places it in all 8 large cages of the sII hydrate
thf_hydrate = hydrate_total.copy()
for i in range(len(large_cages.index)):
    dxyz = thf_center - large_cages[['X','Y','Z']].iloc[i]
    thf_translated = thf.copy()
    thf_translated[['X','Y','Z']] = thf_translated[['X','Y','Z']] - dxyz
    thf_translated['RESIDUE_NUM'] = residue_num
    thf_hydrate = pd.concat([thf_hydrate, thf_translated], ignore_index=True)
    residue_num += 1

thf_hydrate.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ELEMENT      512 non-null    object
 1   X            512 non-null    object
 2   Y            512 non-null    object
 3   Z            512 non-null    object
 4   RESIDUE_NUM  512 non-null    object
 5   MOL          512 non-null    object
dtypes: object(6)
memory usage: 24.1+ KB


In [75]:
thf_hydrate

,ELEMENT,X,Y,Z,RESIDUE_NUM,MOL
0,OW,5.92487,5.92487,5.92487,1,HOH
1,HW1,5.75998,6.83792,5.68955,1,HOH
2,HW2,6.84272,5.77984,5.69518,1,HOH
3,OW,5.32179,5.32179,8.5662,2,HOH
4,HW1,5.53914,5.52632,7.65672,2,HOH
...,...,...,...,...,...,...
507,H08,-0.929,7.885231,-1.290769,144,THF
508,H09,-0.877,9.447231,1.313231,144,THF
509,H0A,-1.497,10.002231,-0.253769,144,THF
510,H0B,1.057,10.591231,0.522231,144,THF


In [76]:
f = open("output/thf_hydrate_unit.pdb", "w")

f.write("CRYST1   17.310   17.310   17.310  90.00  90.00  90.00 P 1           1\n")
for i in range(len(thf_hydrate.index)):
    #Collect PDB variables
    id = str(i)
    atom = thf_hydrate['ELEMENT'].iloc[i]
    resname = thf_hydrate['MOL'].iloc[i]
    resnum = thf_hydrate['RESIDUE_NUM'].iloc[i]
    x = thf_hydrate['X'].iloc[i]
    y = thf_hydrate['Y'].iloc[i]
    z = thf_hydrate['Z'].iloc[i]

    # Create PDB entry
    line = "{:6s}{:5d} {:^4s} {:3s}  {:4d}    {:8.3f}{:8.3f}{:8.3f}{:6.2f}{:6.2f}\n".format(
           "ATOM", i+1, atom, resname, resnum, x, y, z, 0., 0.)
    f.write(line)

f.close()

In [77]:
range(len(thf_hydrate.index))

range(0, 512)

In [78]:
#Make the box 3 x 3
cell_size = 17.30
num_cells = 3

In [79]:
coord_offset = [[-cell_size, 0, cell_size],[0,0,cell_size],[cell_size, 0, cell_size],
                [-cell_size, 0, 0],[cell_size, 0, 0],
                [-cell_size, 0, -cell_size],[0, 0, -cell_size],[cell_size, 0, -cell_size]]

In [80]:
# Create 3x3 plane crystal
thf_hydrate_3_plane = thf_hydrate.copy()
for offset in coord_offset:
    residue_num = thf_hydrate_3_plane['RESIDUE_NUM'].iloc[len(thf_hydrate_3_plane.index) - 1]
    thf_hydrate_unit = thf_hydrate.copy()
    thf_hydrate_unit[['X','Y','Z']] = thf_hydrate_unit[['X','Y','Z']] + offset
    thf_hydrate_unit['RESIDUE_NUM'] = thf_hydrate_unit['RESIDUE_NUM'] + residue_num
    thf_hydrate_3_plane = pd.concat([thf_hydrate_3_plane, thf_hydrate_unit], axis=0, ignore_index=True)

thf_hydrate_3_plane


,ELEMENT,X,Y,Z,RESIDUE_NUM,MOL
0,OW,5.92487,5.92487,5.92487,1,HOH
1,HW1,5.75998,6.83792,5.68955,1,HOH
2,HW2,6.84272,5.77984,5.69518,1,HOH
3,OW,5.32179,5.32179,8.5662,2,HOH
4,HW1,5.53914,5.52632,7.65672,2,HOH
...,...,...,...,...,...,...
4603,H08,16.371,7.885231,-18.590769,1296,THF
4604,H09,16.423,9.447231,-15.986769,1296,THF
4605,H0A,15.803,10.002231,-17.553769,1296,THF
4606,H0B,18.357,10.591231,-16.777769,1296,THF


In [81]:
# Stack 3x3x3 crystal
thf_hydrate_3_3 = thf_hydrate_3_plane.copy()
for i in range(1, num_cells):
    residue_num = thf_hydrate_3_3['RESIDUE_NUM'].iloc[len(thf_hydrate_3_3.index) - 1]
    thf_hydrate_plane_unit = thf_hydrate_3_plane.copy()
    thf_hydrate_plane_unit['Y'] = thf_hydrate_plane_unit['Y'] + (cell_size * i)
    thf_hydrate_plane_unit['RESIDUE_NUM'] = thf_hydrate_plane_unit['RESIDUE_NUM'] + residue_num
    thf_hydrate_3_3 = pd.concat([thf_hydrate_3_3, thf_hydrate_plane_unit], axis=0, ignore_index=True)

thf_hydrate_3_3


,ELEMENT,X,Y,Z,RESIDUE_NUM,MOL
0,OW,5.92487,5.92487,5.92487,1,HOH
1,HW1,5.75998,6.83792,5.68955,1,HOH
2,HW2,6.84272,5.77984,5.69518,1,HOH
3,OW,5.32179,5.32179,8.5662,2,HOH
4,HW1,5.53914,5.52632,7.65672,2,HOH
...,...,...,...,...,...,...
13819,H08,16.371,42.485231,-18.590769,3888,THF
13820,H09,16.423,44.047231,-15.986769,3888,THF
13821,H0A,15.803,44.602231,-17.553769,3888,THF
13822,H0B,18.357,45.191231,-16.777769,3888,THF


In [82]:
f = open("output/thf_hydrate_3_3.pdb", "w")

f.write("CRYST1   52.000   52.000   52.000  90.00  90.00  90.00 P 1           1\n")
for i in range(len(thf_hydrate_3_3.index)):
    #Collect PDB variables
    id = str(i)
    chainid = 'A'
    atom = thf_hydrate_3_3['ELEMENT'].iloc[i]
    resname = thf_hydrate_3_3['MOL'].iloc[i]
    resnum = thf_hydrate_3_3['RESIDUE_NUM'].iloc[i]
    x = thf_hydrate_3_3['X'].iloc[i]
    y = thf_hydrate_3_3['Y'].iloc[i]
    z = thf_hydrate_3_3['Z'].iloc[i]

    # Create PDB entry
    line = "{:6s}{:5d} {:^4s} {:3s} {:1s}{:4d}    {:8.3f}{:8.3f}{:8.3f}{:6.2f}{:6.2f}\n".format(
           "ATOM", i+1, atom, resname, chainid, resnum, x, y, z, 0., 0.)
    f.write(line)

f.close()